# Introduction to Python  

## Pandas - Exploratory Data Analysis (EDA) part 2

In [1]:
import os
import zipfile
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display, Image
from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

## Pandas Data Manipulation: Baby names in the USA

In [2]:
names1880 = pd.read_csv(os.path.join('../Data','names','yob1880.txt'), names=['name', 'sex', 'births'])

FileNotFoundError: [Errno 2] No such file or directory: '../Data/names/yob1880.txt'

In [ ]:
names1880[0:20]

In [ ]:
#names1880.head()
names1880.tail()

In [ ]:
names1880.groupby('sex').births.sum()

In [ ]:
years = range(1880, 2020)
pieces = []
columns = ['name', 'sex', 'births']
for year in years:
    path = os.path.join('../Data','names','yob{}.txt'.format(year))
    frame = pd.read_csv(path, names=columns)
    frame['year'] = year
    pieces.append(frame)
    
# Concatenate everything into a single DataFrame
names = pd.concat(pieces, ignore_index=True)

In [ ]:
names.head(10)

In [ ]:
names.info()

In [ ]:
names.groupby('sex').births.sum()

In [ ]:
total_births = names.pivot_table('births', index='year', columns='sex', aggfunc=sum)

In [ ]:
total_births

In [ ]:
total_births.tail()

In [ ]:
total_births.plot(title='Total births by sex and year', figsize=(12,6));

In [ ]:
def add_prop(group):
    # Integer division floors
    births = group.births.astype(float)
    group['percent'] = births / births.sum()
    return group

names = names.groupby(['year', 'sex']).apply(add_prop)

In [ ]:
#names
names.head()

In [ ]:
names[names.percent > 0.085]

In [ ]:
names[names.name.str.startswith('Stephany')]

In [ ]:
np.allclose(names.groupby(['year', 'sex']).percent.sum(), 1)

In [ ]:
def get_top1000(group):
    return group.sort_values(by='births', ascending=False)[:1000]

grouped = names.groupby(['year', 'sex'])
top1000 = grouped.apply(get_top1000)

In [ ]:
#top1000
pd.options.display.float_format = '{:,.3f}'.format
top1000[:15]

In [ ]:
boys = top1000[top1000.sex == 'M']
girls = top1000[top1000.sex == 'F']

In [ ]:
Walter_names = boys[boys.name=='Walter']
Walter_names[:10]

In [ ]:
top1000.drop(['sex','year'], axis=1, inplace=True)
total_births_top1000 = top1000.pivot_table('births', index='year', columns='name', aggfunc=sum)

In [ ]:
total_births_top1000.head()

In [ ]:
subset = total_births_top1000[['John', 'Harry', 'Mary', 'Marilyn']]
subset.plot(subplots=True, figsize=(12, 12), grid=False,
title="Number of births per year")

In [ ]:
table = top1000.pivot_table('percent', index='year', columns='sex', aggfunc=sum)

table.plot(title='Sum of table1000.percent by year and sex', 
           yticks=np.linspace(0, 1.2, 13), 
           xticks=range(1880, 2020, 10), 
           figsize=(12,8))

In [ ]:
df = boys[boys.year == 2019]
prop_cumsum = df.sort_values(by='percent', ascending=False).percent.cumsum()

In [ ]:
def get_quantile_count(group, q=0.5):
    group = group.sort_values(by='percent', ascending=False)
    return group.percent.cumsum().values.searchsorted(q) + 1

prop_cumsum.values.searchsorted(0.5)

In [ ]:
diversity = top1000.groupby(['year', 'sex']).apply(get_quantile_count)
diversity = diversity.unstack('sex')
diversity.head()

In [ ]:
diversity.plot(title="Number of popular names in top 50%", figsize=(12,8))

In [ ]:
# extract last letter from name column

get_last_letter = lambda x: x[-1]
last_letters = names['name'].map(get_last_letter)

last_letters.name = 'last_letter'

table = names.pivot_table('births', index=last_letters, columns=['sex', 'year'], aggfunc=sum)

subtable = table.reindex(columns=[1910, 1960, 2010], level='year')
subtable.head()

In [ ]:
subtable.sum()

In [ ]:
letter_prop = subtable / subtable.sum().astype(float)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 8))
letter_prop['M'].plot(kind='bar', rot=0, ax=axes[0], title='Male')
letter_prop['F'].plot(kind='bar', rot=0, ax=axes[1], title='Female',legend=False)

In [ ]:
letter_prop = table / table.sum().astype(float)
dny_ts = letter_prop.loc[['d', 'n', 'y'], 'M'].T
dny_ts.head()

In [ ]:
letter_prop

In [ ]:
dny_ts.plot(figsize=(12,8))

In [ ]:
all_names = top1000.name.unique()
mask = np.array(['lesl' in x.lower() for x in all_names])
lesley_like = all_names[mask]
lesley_like

In [ ]:
filtered = top1000[top1000.name.isin(lesley_like)]
filtered.groupby('name').births.sum()

In [ ]:
table = filtered.pivot_table('births', index='year', columns='sex', aggfunc='sum')
table = table.div(table.sum(1), axis=0)
table.tail()

In [ ]:
table.plot(style={'M': 'k-', 'F': 'k--'}, figsize=(12,8))